<a href="https://colab.research.google.com/github/crgg1985/ECOSISTEMA1/blob/main/Extractor_y_Conversor_de_Reglas_a_MQL4_19_11_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Introducción

Este Google Colab forma parte del material adicional que VTALGO Academy regala a sus alumnos y que tiene como finalidad automatizar y hacer más fácil el proceso de generación de sistemas según el Ecosistema 1.

Aquí simplemente, se cargan las librearías necesarias.

In [ ]:
# Importación de librerías
import os
import io
import json
import re
from typing import List, Literal, Dict, Tuple
import pandas as pd
from google.colab import files
import textwrap

# 2. Subir archivo de extracción a Google Colab para su posterior tratamiento

Los pasos a seguir en este apartado son:

1. Subir el archivo de salida de WEKA (según clase `VAc_DMRI_Eco1_WekaAvanzado` diponible en Vimeo). Aquí están todas las reglas obtenidas al aplicar las diferentes semillas.
2. Una vez subido, se ejecuta el código definido en la función `main` y se obtendrá un archivo con las reglas extraídas del archivo de texto.
3. Se deberá además de disponer de un archivo con formato `json` en el que se mapeará cada nombre de los features extraídos (tal y como estén en el archivo del extractor, en el encabezado, primera fila) con el código MQL que calcula el feature.

Esta es la parte donde se sube el archivo tal y como sale de WEKA.

In [ ]:
# Sube el archivo correspondiente a las reglas en crudo desde tu computadora
raw_rules = files.upload()

# El archivo estará disponible en la ruta actual de Colab
# Si subes "ReglasMasivas.txt", lo puedes referenciar directamente como:
filename = list(raw_rules.keys())[0]

Saving salida_masiva_20240913120320.txt to salida_masiva_20240913120320.txt


Lo siguiente, son dos funciones que ayudan a extraer la parte de las reglas. La función `extract_rules` tiene un argumento: `target`que permite definir si la extracción tenía como objetivo obtener reglas alcistas o bajistas.

In [ ]:
def extract_rules(filename: str, target: Literal['UP', 'DOWN']) -> List[str]:
    """
    Extrae reglas específicas de un archivo de texto.

    Esta función lee un archivo de texto, busca reglas JRIP y extrae aquellas
    que coinciden con el target especificado (UP o DOWN).

    Args:
        filename (str): El nombre del archivo a leer.
        target (Literal['UP', 'DOWN']): El tipo de reglas a extraer ('UP' o 'DOWN').

    Returns:
        List[str]: Una lista de reglas que coinciden con el target especificado.

    Raises:
        ValueError: Si el target no es 'UP' o 'DOWN'.
        FileNotFoundError: Si el archivo especificado no se encuentra.
        Exception: Para cualquier otro error inesperado.
    """
    # Verificar que el target sea válido
    if target not in ['UP', 'DOWN']:
        raise ValueError("El target debe ser 'UP' o 'DOWN'")

    # Leer el archivo
    with open(filename, 'r') as file:
        content = file.read()

    # Definir el patrón para extraer las reglas
    pattern = r'JRIP rules:\n===========\n(.*?)\n\nNumber of Rules :'

    # Encontrar todas las coincidencias
    matches = re.findall(pattern, content, re.DOTALL)

    # Lista para almacenar todas las reglas
    all_rules: List[str] = []

    # Procesar cada coincidencia
    for match in matches:
        # Dividir las reglas individuales
        rules = match.strip().split('\n')
        # Añadir las reglas a la lista, eliminando líneas vacías y filtrando por target
        all_rules.extend([rule.strip() for rule in rules if rule.strip() and f'LABEL={target}' in rule])

    return all_rules

def save_rules_to_file(rules: List[str], output_filename: str, include_rule_no: bool = False) -> None:
  """
  Guarda las reglas extraídas en un archivo y lo descarga en Google Colab.

  Args:
      rules (List[str]): Lista de reglas a guardar.
      output_filename (str): Nombre del archivo de salida.
  """
  # Crear un objeto StringIO para escribir las reglas
  output = io.StringIO()
  for i, rule in enumerate(rules, 1):
    if include_rule_no:
      output.write(f"{i}. {rule}\n")
    output.write(f"{rule}\n")

  # Guardar el archivo en Colab
  with open(output_filename, 'w') as f:
      f.write(output.getvalue())

  # Descargar el archivo
  files.download(output_filename)

La siguiente función ejecuta todo el código anterior y genera un archivo para las reglas extraídas.

In [ ]:
def main() -> None:
    """
    Función principal para demostrar el uso de extract_rules y save_rules_to_file.
    """
    target: Literal['UP', 'DOWN'] = 'UP'  # o 'DOWN'
    output_filename = f'reglas_extraidas_{target}.txt'

    try:
        extracted_rules = extract_rules(filename, target)

        print(f"Reglas extraídas para LABEL={target}:")
        for i, rule in enumerate(extracted_rules, 1):
            print(f"{i}. {rule}")

        print(f"\nTotal de reglas extraídas: {len(extracted_rules)}")

        # Guardar las reglas extraídas
        save_rules_to_file(extracted_rules, output_filename)
        total_rules = len(extracted_rules)
        print(f"\nLas {total_rules} reglas han sido guardadas en '{output_filename}' y se iniciará la descarga automáticamente.")

    except FileNotFoundError:
        print(f"Error: No se pudo encontrar el archivo '{filename}'")
    except ValueError as e:
        print(f"Error: {str(e)}")
    except Exception as e:
        print(f"Ocurrió un error inesperado: {str(e)}")

if __name__ == "__main__":
    main()

Reglas extraídas para LABEL=UP:
1. (HH_22 <= 0.985) => LABEL=UP (614.0/306.0)
2. (ATR_28 >= 0.7) and (EMA_200 <= 82.955) and (HH_29 <= 1.006) => LABEL=UP (166.0/65.0)
3. (DMIPlus_100 <= 17.525) and (BearsPower_3 >= -0.372) => LABEL=UP (257.0/114.0)
4. (BearsPower_3 >= -0.273) and (IBS_5 >= 0.847) => LABEL=UP (174.0/79.0)
5. (CKC_7_25_25 <= 0.815) and (IBS_9 <= 0.632) and (Stoch_24_18_18 <= 29.754) => LABEL=UP (163.0/74.0)
6. (Mom_100 <= 97.384) and (ATR_200 >= 0.898) => LABEL=UP (325.0/150.0)
7. (RSI_200 <= 51.298) and (Highest_14 <= 18.452) and (SuperTrend_28_2.0 <= 94.599) => LABEL=UP (323.0/149.0)
8. (IBS_5 >= 0.481) and (IBS_17 >= 0.44) and (BBR_BB_Up_7_2_7 <= 1.002) => LABEL=UP (239.0/113.0)
9. (HH_29 <= 1.005) and (SuperTrend_28_2.0 <= 90.849) => LABEL=UP (382.0/173.0)
10. (CL <= 1.001) and (HH_8 <= 0.992) and (BBR_BB_Dwn_50_2_3 <= 0.997) => LABEL=UP (67.0/17.0)
11. (MACD_24_32_9 >= 0.11) and (VROC_50 >= 200) => LABEL=UP (162.0/75.0)
12. (HH_22 <= 0.985) => LABEL=UP (614.0/306.0)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


Las 985 reglas han sido guardadas en 'reglas_extraidas_UP.txt' y se iniciará la descarga automáticamente.


#3. Carga del Diccionario de Features y transformación de las reglas.

En este paso, se carga primero el archivo con formato `json` que tiene el "mapeo" entre el encabezado y el código MQL que genera el feature para cada vela.

## 3.1. Carga del diccionario
El proceso parte de la carga del archivo con el mapeo, que deja en el Colab un diccionario para su uso posterior por las funciones de mapeado y parseado de reglas.

In [ ]:
# Subimos el diccionario
uploaded = files.upload()

Este pequeño código cuenta el número de features que contiene el diccionario, para comprobar que tenemos correctamente todo bien para poder traducir.

In [ ]:
# Funciones para cargarlo en el Colab para poder utilizarlo

def load_config(file_content, file_type='json'):
    """
    Carga la configuración desde un contenido de archivo JSON o YAML.

    Args:
        file_content (str): Contenido del archivo de configuración.
        file_type (str): Tipo de archivo ('yaml' o 'json').

    Returns:
        dict: Diccionario con la configuración cargada.
    """
    try:
        if file_type == 'yaml':
            import yaml
            return yaml.safe_load(file_content)
        elif file_type == 'json':
            return json.loads(file_content)
        else:
            raise ValueError("Tipo de archivo no soportado. Use 'yaml' o 'json'.")
    except Exception as e:
        print(f"Error al cargar la configuración: {str(e)}")
        return {}

# Cargar la configuración
if uploaded:
    file_name = next(iter(uploaded))
    file_content = uploaded[file_name].decode('utf-8')
    header_to_mql4 = load_config(file_content, 'json')

    print("Configuración cargada exitosamente.")
    print("Primeras 5 entradas del diccionario header_to_mql4:")
    for i, (key, value) in enumerate(header_to_mql4.items()):
        if i >= 5:
            break
        print(f"{key}: {value}")
else:
    print("No se subió ningún archivo.")

In [ ]:
with open('header_to_mql4.json', 'r') as f:
    data = json.load(f)

print(f"Número total de entradas: {len(data)}")

## 3.2. Transformación de las reglas

La transformación de las reglas consiste en: tomar las reglas una a una del archivo y realizar los siguientes pasos:

* Primero, separar la regla de sus métricas, que están separadas por => LABEL=UP/DOWN (a/b). Las reglas a la izquierda, las métricas a la derecha.
* Después, con las métricas, se debe crear un dataframe que tenga como columnas, el identificador de las reglas y el significado de las métricas, tal y como las arroja WEKA.
* Con las reglas, hay que quitarle los paréntesis de las condiciones (separadas por "and"). Quedará algo como condicion operador valor (donde operador será >=, =, <=...).
* Usar el diccionario para sustituir condicion por el código MQL4 correspondiente.
* Rehacer la condición, ya que con código MQL4 y encerrar cada una entre paréntesis y sustituir los "and" por "&&" y los "or" (si los hubiera) por "||) que es como se usan en MQL4.

Ejemplo:
Partimos de la regla del archivo de reglas.txt:

`(HH_4 <= 0.992) and (BBR_Up_Dwn_37_3 >= 1.08) and (DMIMinus_100 <= 19.269) and (IBS_17 >= 0.342) => LABEL=UP (120.0/39.0)`

El resultado final será:

`(SafeDivide(High[1], High[1 + 4]) <= 0.992) && (SafeDivide(iBands(NULL, 0, 37, 3.0, 0, PRICE_CLOSE, MODE_UPPER, 1), iBands(NULL, 0, 37, 3.0, 0, PRICE_CLOSE, MODE_LOWER, 1)) >= 1.08) && (iADX(NULL, 0, 100, PRICE_CLOSE, MODE_MINUSDI, 1) <= 19.269) && (ibs(1+17) >= 0.342)`

Para la regla. Y para la métrica debería haber un dataframe con los datos.

Las siguientes funciones se encargan de procesar las reglas, de acuerdo a las condiciones marcadas y guardarlas en un archivo txt:

In [ ]:
def convert_condition(condition: str, dictionary: Dict[str, str]) -> str:
    """
    Convierte una condición individual usando el diccionario proporcionado.

    Args:
        condition (str): La condición a convertir.
        dictionary (Dict[str, str]): El diccionario para la conversión.

    Returns:
        str: La condición convertida.
    """
    parts = condition.split()
    if len(parts) == 3:
        variable, operator, value = parts
        if variable in dictionary:
            return f"({dictionary[variable]} {operator} {value})"
    return condition

def process_rule(rule: str, dictionary: Dict[str, str]) -> Tuple[str, str, float, float, float]:
    """
    Procesa una regla completa, separando la regla de sus métricas y convirtiendo las condiciones.

    Args:
        rule (str): La regla completa a procesar.
        dictionary (Dict[str, str]): El diccionario para la conversión de condiciones.

    Returns:
        Tuple[str, str, float, float, float]: Una tupla que contiene la regla procesada, la etiqueta y las métricas detalladas.
    """
    # Separar la regla de sus métricas
    rule_part, metrics_part = rule.split('=>')

    # Procesar la parte de la regla
    conditions = re.findall(r'\((.*?)\)', rule_part)
    processed_conditions = [convert_condition(cond, dictionary) for cond in conditions]
    processed_rule = ' && '.join(processed_conditions)

    # Procesar la parte de las métricas
    label_part, metrics = metrics_part.split('(')
    label = label_part.split('=')[1].strip()
    a, b = map(float, metrics.strip(')').split('/'))

    instancias_cubiertas = a
    instancias_misclasificadas = b
    instancias_correctas = a - b

    return processed_rule, label, instancias_cubiertas, instancias_misclasificadas, instancias_correctas

def save_rules_to_file(rules: List[str], file_path: str) -> None:
    """
    Guarda las reglas procesadas en un archivo de texto.

    Args:
        rules (List[str]): Lista de reglas procesadas para guardar.
        file_path (str): Ruta del archivo donde se guardarán las reglas.
    """
    with open(file_path, 'w') as f:
        for i, rule in enumerate(rules, 1):
            f.write(f"Rule_{i}: {rule}\n")
    print(f"Reglas guardadas en {file_path}")

In [ ]:
def main():
    """
    Función principal que ejecuta el proceso de conversión de reglas WEKA a MQL4.
    """
    dictionary = header_to_mql4
    # Leer el archivo de reglas
    with open('reglas_extraidas_UP.txt', 'r') as f:
        rules = f.readlines()

    # Procesar cada regla
    processed_rules: List[str] = []
    metrics_data: List[Dict[str, str]] = []

    for i, rule in enumerate(rules, 1):
        processed_rule, label, instancias_cubiertas, instancias_misclasificadas, instancias_correctas = process_rule(rule.strip(), dictionary)
        processed_rules.append(processed_rule)
        metrics_data.append({
            'Rule_ID': f'Rule_{i}',
            'Label': label,
            'instancias_cubiertas': int(instancias_cubiertas),
            'instancias_misclasificadas': int(instancias_misclasificadas),
            'instancias_correctas': int(instancias_correctas)
        })

    # Crear DataFrame con las métricas
    df_metrics = pd.DataFrame(metrics_data)

    # Imprimir las reglas procesadas y el DataFrame de métricas
    print("Reglas procesadas:")
    for i, rule in enumerate(processed_rules, 1):
        print(f"Rule_{i}: {rule}")

    print("\nDataFrame de métricas:")
    print(df_metrics)

    # Guardar las reglas procesadas en un archivo
    save_rules_to_file(processed_rules, 'processed_rules.txt')

    # Guardar el DataFrame de métricas en un archivo CSV
    df_metrics.to_csv('metrics.csv', index=False)
    print("Métricas guardadas en metrics.csv")

if __name__ == "__main__":
    main()

#4. Generación del código MQL4

In [ ]:
def balance_parentheses(condition: str) -> str:
    """
    Balancea los paréntesis en una condición dada.

    Args:
        condition (str): La condición a balancear.

    Returns:
        str: La condición con paréntesis balanceados.
    """
    open_count = condition.count('(')
    close_count = condition.count(')')
    if open_count > close_count:
        condition += ')' * (open_count - close_count)
    return condition

def fix_safe_divide(condition: str) -> str:
    """
    Corrige las llamadas a SafeDivide en una condición dada.

    Args:
        condition (str): La condición a corregir.

    Returns:
        str: La condición con las llamadas a SafeDivide corregidas.
    """
    safe_divide_calls = re.findall(r'SafeDivide\((.*?)\)', condition)

    for call in safe_divide_calls:
        comma_count = 0
        nested_level = 0
        for char in call:
            if char == '(':
                nested_level += 1
            elif char == ')':
                nested_level -= 1
            elif char == ',' and nested_level == 0:
                comma_count += 1

        if comma_count > 1:
            args = re.split(r',\s*(?=[^()]*(?:\(|$))', call)
            corrected_call = f"SafeDivide({args[0]}, {args[1]})"
            condition = condition.replace(f"SafeDivide({call})", corrected_call)

    return condition

def generate_mql4_rules(processed_rules: List[str], direction: Literal['UP', 'DOWN']) -> str:
    """
    Genera código MQL4 a partir de las reglas procesadas.

    Args:
        processed_rules (List[str]): Lista de reglas procesadas.
        direction (Literal['UP', 'DOWN']): Dirección de las reglas (compra o venta).

    Returns:
        str: Código MQL4 generado.
    """
    mql4_code = """
//+------------------------------------------------------------------+
//|                                                 rules_tester.mq4 |
//|                                      Juan José Expósito González |
//|                                          VTALGO ACADEMY (C) 2024 |
//+------------------------------------------------------------------+
#property copyright "VTALGO ACADEMY (C) 2024"
#property version   "1.00"
#property strict

//+------------------------------------------------------------------+
//| IMPORTAMOS LIBRERÍAS                                             |
//+------------------------------------------------------------------+
#include <Toolbox/VAC_Toolbox.mqh>

//+------------------------------------------------------------------+

"""

    for i, rule_condition in enumerate(processed_rules, 1):
        function_name = f"Rule{'Buy' if direction == 'UP' else 'Sell'}_{i}"
        rule_condition = fix_safe_divide(rule_condition)
        rule_condition = balance_parentheses(rule_condition)

        mql4_code += f"""
bool {function_name}() {{
    return ({rule_condition});
}}
"""

    mql4_code += f"""
// Función maestra para ejecutar las reglas
bool Execute{'Buy' if direction == 'UP' else 'Sell'}Rule(int ruleNumber) {{
    switch(ruleNumber) {{
"""

    for i in range(1, len(processed_rules) + 1):
        mql4_code += f"        case {i}: return Rule{'Buy' if direction == 'UP' else 'Sell'}_{i}();\n"

    mql4_code += """
        default: return false;
    }
}
"""

    mql4_code += f"""
// Dummy code
bool ExecuteBuyRules(){{ return false; }}
bool ExecuteSellRules(){{ return false; }}
  """

    return mql4_code

def save_mql4_code(mql4_code: str, direction: Literal['UP', 'DOWN'], output_file: str = None) -> str:
    """
    Guarda el código MQL4 generado en un archivo.

    Args:
        mql4_code (str): Código MQL4 generado.
        direction (Literal['UP', 'DOWN']): Dirección de las reglas (compra o venta).
        output_file (str, optional): Nombre del archivo de salida. Si es None, se genera automáticamente.

    Returns:
        str: Nombre del archivo donde se guardó el código.
    """
    if output_file is None:
        output_file = f"TradingRules{'Buy' if direction == 'UP' else 'Sell'}.mqh"

    with open(output_file, 'w') as file:
        file.write(mql4_code)

    return output_file

In [ ]:
def main() -> None:
    """
    Función principal que ejecuta el proceso de conversión de reglas WEKA a MQL4.
    """
    dictionary = header_to_mql4

    with open('reglas_extraidas_UP.txt', 'r') as f:
        rules = f.readlines()

    processed_rules: List[str] = []
    metrics_data: List[Dict[str, any]] = []

    for i, rule in enumerate(rules, 1):
        processed_rule, label, total_cases, misclassified_cases, extended_stats = process_rule(rule.strip(), dictionary)
        processed_rules.append(processed_rule)

    print("Reglas procesadas:")
    for i, rule in enumerate(processed_rules, 1):
        print(f"Rule_{i}: {rule}")

    save_rules_to_file(processed_rules, 'processed_rules.txt')


    # Generar y guardar código MQL4
    mql4_code = generate_mql4_rules(processed_rules, 'UP')  # Asumimos que todas las reglas son para compra
    output_file = save_mql4_code(mql4_code, 'UP')
    print(f"Código MQL4 generado y guardado en {output_file}")

if __name__ == "__main__":
    main()

#5. Selección de reglas tras la primera optimización

Una vez que hemos realizado la optimización para testear las reglas una a una, el siguiente código las selecciona y las guarda en un archivo, para proceder después a combinarlas.texto en negrita

Esta parte, sube el archivo de resultados del backtest y selecciona aquéllas reglas que cumplen los criterios de validación (cortería de Javi @Javords).

In [ ]:
# Sube el archivo correspondiente a los resultados del backtest de las reglas
raw_backtest_results = files.upload()

# Extraer el nombre del archivo subido (Colab guarda los archivos subidos en un diccionario)
filename = list(raw_backtest_results.keys())[0]

In [ ]:
# Leer el archivo CSV
df_back_results = pd.read_csv(filename)

# Crear una nueva columna 'ID' con números consecutivos
df_back_results['ID'] = range(1, len(df_back_results) + 1)

# Filtrar por Profit Factor (PF) mayor o igual a 1.8
pf_minimo = 1.8
df_back_results_fil1 = df_back_results[df_back_results['PF'] >= pf_minimo]

# Filtrar por número mínimo de operaciones (Num_Ops) mayor o igual a 150
num_ops_minimo = 150
df_back_results_fil2 = df_back_results_fil1[df_back_results_fil1['Num_Ops'] >= num_ops_minimo]

# Filtrar por porcentaje mínimo de ganadoras (PctWin) mayor o igual a 60%
pctwin_minimo = 60
df_filtrado_final = df_back_results_fil2[df_back_results_fil2['PctWin'] >= pctwin_minimo]

# Extraer los IDs de las filas filtradas como una lista de enteros
ids_resultantes = df_filtrado_final['ID'].tolist()

# Unir los IDs como una cadena de texto separada por comas, pero sin comillas
ids_txt = ",".join(map(str, ids_resultantes))

print(ids_resultantes)

El siguiente código es el que permite crear el archivo MQL con las reglas seleccionadas.

In [ ]:
def select_rules(input_file: str, output_file: str, rule_indices: list[int]) -> bool:
    """
    Selecciona reglas específicas de un archivo de entrada y las escribe en un archivo de salida.

    Args:
    input_file (str): Nombre del archivo que contiene todas las reglas.
    output_file (str): Nombre del archivo donde se guardarán las reglas seleccionadas.
    rule_indices (list[int]): Lista de índices de reglas a seleccionar (comenzando desde 1).

    Returns:
    bool: True si el proceso se completó con éxito, False en caso contrario.
    """
    try:
        # Ordenar los índices de menor a mayor
        rule_indices.sort()

        with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
            rules = infile.readlines()

            for index in rule_indices:
                if 1 <= index <= len(rules):
                    # outfile.write(f"Rule_{index}: {rules[index-1]}")
                    rule = rules[index-1].split(":")[-1]
                    outfile.write(f"{rule}")
                else:
                    print(f"Advertencia: El índice {index} está fuera de rango.")

        print(f"Reglas seleccionadas guardadas en {output_file}")
        return True

    except IOError as e:
        print(f"Error al procesar los archivos: {e}")
        return False

Aquí utilizamos el código de arriba para seleccionar las reglas que cumplen los criterios.

In [ ]:
rules_to_select = ids_resultantes
def main() -> None:
    input_filename = "processed_rules.txt"
    output_filename = "SelectedRules.txt"


    success = select_rules(input_filename, output_filename, rules_to_select)
    if success:
        print("Proceso completado con éxito.")
    else:
        print("Hubo un error en el proceso.")

if __name__ == "__main__":
  main()

#6. Combinación de reglas seleccionadas

In [ ]:
def generate_mql4_rules(input_file: str, output_file: str, direction: Literal['UP', 'DOWN']) -> bool:
    """
    Genera un archivo .mqh con funciones MQL4 para cada regla seleccionada,
    incluyendo funciones auxiliares y parámetros de entrada para optimización.

    Args:
    input_file (str): Nombre del archivo que contiene las reglas seleccionadas.
    output_file (str): Nombre del archivo .mqh de salida.
    direction (Literal['UP', 'DOWN']): Dirección de las reglas (compra o venta).

    Returns:
    bool: True si el proceso se completó con éxito, False en caso contrario.
    """
    try:
        with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
            rules = infile.readlines()

            # Escribir encabezado del archivo
            outfile.write("//+------------------------------------------------------------------+\n")
            outfile.write("//|                                                SelectedRules.mqh |\n")
            outfile.write("//|                                      Juan José Expósito González |\n")
            outfile.write("//|                                          VTALGO ACADEMY (C) 2024 |\n")
            outfile.write("//+------------------------------------------------------------------+\n\n")

            # Añadir funciones auxiliares
            outfile.write("///+------------------------------------------------------------------+\n")
            outfile.write("//| IMPORTAMOS LIBRERÍAS                                             |\n")
            outfile.write("//+------------------------------------------------------------------+\n")
            outfile.write("#include <Toolbox/VAC_Toolbox.mqh>\n")
            outfile.write("//+------------------------------------------------------------------+\n\n")


            # Añadir parámetros de entrada
            valid_rule_count = sum(1 for rule in rules if rule.strip())
            outfile.write(f"sinput string rule_combiner = \"===================== COMBINACIÓN DE REGLAS =====================\";\n")
            for i in range(valid_rule_count):
                outfile.write(f"input int Rule{i} = 1; // Activar/Desactivar Regla {i} (1 = Activada, 0 = Desactivada)\n")
            outfile.write("\n")

            # Generar funciones para cada regla
            valid_rule_count = 0
            for i, rule in enumerate(rules):
                rule = rule.strip()
                if not rule:  # Saltar reglas vacías
                    print(f"Advertencia: Regla {i+1} está vacía. Saltando.")
                    continue

                function_name = f"Rule{'Buy' if direction == 'UP' else 'Sell'}_{valid_rule_count}"

                outfile.write(f"bool {function_name}() {{\n")
                outfile.write(f"    return ({rule});\n")
                outfile.write("}\n\n")

                valid_rule_count += 1

            # Generar función para ejecutar combinación de reglas
            outfile.write(f"bool Execute{'Buy' if direction == 'UP' else 'Sell'}Rules() {{\n")
            outfile.write("    bool result = false;\n")
            for i in range(valid_rule_count):
                outfile.write(f"    if (Rule{i} == 1) {{\n")
                outfile.write(f"        result = result || Rule{'Buy' if direction == 'UP' else 'Sell'}_{i}();\n")
                outfile.write("    }\n")
            outfile.write("    return result;\n")
            outfile.write("}\n")

            outfile.write("//Dummy code\n")
            outfile.write("bool ExecuteBuyRule(int ruleNumber) { return false; }")


        print(f"Archivo MQL4 generado y guardado en {output_file}")
        print(f"Total de reglas válidas procesadas: {valid_rule_count}")
        return True

    except IOError as e:
        print(f"Error al procesar los archivos: {e}")
        return False

In [ ]:
# Ejemplo de uso
if __name__ == "__main__":
    input_filename = "SelectedRules.txt"
    output_filename = "SelectedRules.mqh"
    direction = 'UP'  # o 'DOWN' según sea necesario

    success = generate_mql4_rules(input_filename, output_filename, direction)
    if success:
        print("Proceso completado con éxito.")
    else:
        print("Hubo un error en el proceso.")

# [ANEXO] Generación del archivo para la optimización de la combinación de reglas

In [ ]:
def generar_opti_comb(num_reglas: int) -> str:
  """

  """
  comb_str: str = ""
  for i in range(num_reglas):
    comb_str += f"Rule{i}=1\nRule{i},F=1\nRule{i},1=0\nRule{i},2=1\nRule{i},3=1\n"
  return comb_str


rules_comb = generar_opti_comb(len(rules_to_select))

common_text = f"""sz==============================
Lots=0.10000000
Lots,F=0
Lots,1=0.10000000
Lots,2=0.00000000
Lots,3=0.00000000
MagicNumber=10
MagicNumber,F=0
MagicNumber,1=-1
MagicNumber,2=0
MagicNumber,3=0
Comentario=VAC_RULES_TESTER_PLANTILLA
MaxOrdenesAbiertas=1
MaxOrdenesAbiertas,F=0
MaxOrdenesAbiertas,1=1
MaxOrdenesAbiertas,2=0
MaxOrdenesAbiertas,3=0
MaxSpread=3.00000000
MaxSpread,F=0
MaxSpread,1=2.00000000
MaxSpread,2=0.00000000
MaxSpread,3=0.00000000
MaxSlippage=1.00000000
MaxSlippage,F=0
MaxSlippage,1=1.00000000
MaxSlippage,2=0.00000000
MaxSlippage,3=0.00000000
CadaVela=1
CadaVela,F=0
CadaVela,1=0
CadaVela,2=1
CadaVela,3=1
sext001===================> MODO DE SALIDA <=========================
exit_mode=1
exit_mode,F=0
exit_mode,1=0
exit_mode,2=0
exit_mode,3=0
N_Candles=1
N_Candles,F=0
N_Candles,1=1
N_Candles,2=0
N_Candles,3=0
testt001===================> MODO DE TESTEO <=========================
test_mode=1
test_mode,F=0
test_mode,1=0
test_mode,2=0
test_mode,3=0
rule_combiner====================== COMBINACIÓN DE REGLAS =====================
{rules_comb}srule_tester=================> TEST RULES ONE BY ONE <================
test_rule=1
test_rule,F=0
test_rule,1=1
test_rule,2=1
test_rule,3=356
FO_OPTIM=================================> FUNCIONES OBJETIVO <================================
FO_CODES = "0: NP. 1: NPxPF^2. 2: NP^2xPF. 3: SQN, 4: K-Ratio, 5: Sortino, 6: Sharpe, 7: Stagnation"
FO=1
FO,F=0
FO,1=0
FO,2=0
FO,3=0"""

def generate_combi_file(output_file: str, common_text: str) -> None:
  with open(output_file, 'w') as file:
    file.write(common_text)
  print(f"Archivo generado: {output_file}")

generate_combi_file('OPTI_COMBI.set', common_text)